In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint
from matplotlib import rcParams
import os

WORK_DIR = os.getcwd()

In [43]:
# Initial temperature (Historical)
def get_mean_temperature(lat, lon, start_year=1850, end_year=2014, output_csv='initial_temperature.csv'):
    # Open the NetCDF files

    temp_hist =xr.open_dataset(f"{WORK_DIR}/Climate_data/Historical/tas_Amon_FGOALS-f3-L_historical_r1i1p1f1_gr_185001-201412.nc")

    # Extract the 'tas' variable
    temp_k = temp_hist['tas']

    # Calculate the monthly mean temperature from start_year to end_year
    temp_filtered = temp_k.sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))
    monthly_means = temp_filtered.groupby('time.month').mean(dim='time')

    # Change the coordinates from 0-360 to -180 to 180
    monthly_means = monthly_means.assign_coords(lon=(monthly_means.lon + 180) % 360 - 180)
    monthly_means = monthly_means.sortby('lon')

    # Select the temperature value for the particular region
    temp_region = monthly_means.sel(lat=lat, lon=lon, method='nearest')

    # Convert to a DataFrame
    df2 = temp_region.to_dataframe()
    df2.drop(['lat', 'lon','height'], inplace=True, axis=1)

    # Calculate the number of months in the specified period
    num_months = (end_year - start_year + 1) * 12

    # Repeat the monthly means for the specified period
    repeated_monthly_means = pd.DataFrame(np.tile(df2.values, (num_months // 12, 1)), columns=['T'])

    # Create a DateTimeIndex starting from 'start_year-01-01' with monthly frequency
    start_date = f'{start_year}-01-01'
    repeated_monthly_means.index = pd.date_range(start=start_date, periods=num_months, freq='MS')

    # Save the DataFrame as a CSV file
    repeated_monthly_means.to_csv(output_csv, index=True)

    # Load the CSV file to display specific parts
    full_df = pd.read_csv(output_csv, index_col=0, parse_dates=True)

    return full_df


def run_for_multiple_locations(csv_file, start_year=1850, end_year=2014):
    # Read the CSV file into a DataFrame
    locations_df = pd.read_csv(csv_file)
    
    for _, row in locations_df.iterrows():
        lat = row['Latitude']
        lon = row['Longitude']
        name = row['Network ID']
        output_csv = f"{WORK_DIR}/Temperature_monthly/Temp_monthly/initial_temp/FGOALS_{name}_SSP2.csv"
        print(f"Processing {name} at lat: {lat}, lon: {lon}")
        get_mean_temperature(lat, lon, start_year, end_year, output_csv)

# Run the function for all locations in the CSV file
csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
run_for_multiple_locations(csv_file)

Processing M_PL_040 at lat: 18.35, lon: -77.65
Processing M_PL_030 at lat: 8.93, lon: -67.41
Processing M_PL_035 at lat: 17.92, lon: -76.19
Processing M_PL_026 at lat: -0.5, lon: -90.5
Processing M_PL_012 at lat: 28.13, lon: -17.25
Processing M_PL_058 at lat: 42.29, lon: 3.23
Processing M_PL_050 at lat: 28.27, lon: -16.61
Processing M_PL_007 at lat: 52.41, lon: 1.09
Processing M_PL_072_02 at lat: -37.73, lon: -58.23
Processing M_PL_051 at lat: -41.07, lon: -71.52
Processing M_PL_043 at lat: 56.24, lon: 9.95


In [44]:
#1850 to 2014 temperature (monthly)

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

def process_temperature(lat, lon, start_year=1850, end_year=2014, output_csv='temperature.csv'):
    # Load datasets
    temp_hist =xr.open_dataset(f"{WORK_DIR}/Climate_data/Historical/tas_Amon_FGOALS-f3-L_historical_r1i1p1f1_gr_185001-201412.nc")
  
    # Extract the 'tas' variable
    temp_k = temp_hist['tas']
    
    # Clip the data for the specified period
    temp_filtered = temp_k.sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))
    
    # Change the coordinates from 0-360 to -180 to 180
    temp_filtered = temp_filtered.assign_coords(lon=(temp_filtered.lon + 180) % 360 - 180)
    temp_filtered = temp_filtered.sortby('lon')
    
    # Select the temperature value for the particular region
    temp_region = temp_filtered.sel(lat=lat, lon=lon, method='nearest')
    
    # Convert to a DataFrame
    df2 = temp_region.to_dataframe().reset_index()
    
    # Drop unnecessary columns
    df2.drop(['lat', 'lon', 'time','height'], axis=1, inplace=True)
    df2.rename(columns={'tas': 'T'}, inplace=True)
    
    # Create a DateTimeIndex starting from 'start_year-01-01' with monthly frequency
    start_date = f'{start_year}-01-01'
    df2.index = pd.date_range(start=start_date, periods=len(df2), freq='MS')
    
    # Save the resulting DataFrame as a CSV file
    df2.to_csv(output_csv, index=True)
    
    return df2

def run_for_multiple_locations(csv_file, start_year=1850, end_year=2014):
    # Read the CSV file into a DataFrame
    locations_df = pd.read_csv(csv_file)
    
    for _, row in locations_df.iterrows():
        lat = row['Latitude']
        lon = row['Longitude']
        name = row['Network ID']
        output_csv = f"{WORK_DIR}/Temperature_monthly/Temp_monthly/1850_2014_temp/FGOALS_{name}_SSP2.csv"
        print(f"Processing {name} at lat: {lat}, lon: {lon}")
        process_temperature(lat, lon, start_year, end_year, output_csv)

# Run the function for all locations in the CSV file
csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
run_for_multiple_locations(csv_file)


Processing M_PL_040 at lat: 18.35, lon: -77.65
Processing M_PL_030 at lat: 8.93, lon: -67.41
Processing M_PL_035 at lat: 17.92, lon: -76.19
Processing M_PL_026 at lat: -0.5, lon: -90.5
Processing M_PL_012 at lat: 28.13, lon: -17.25
Processing M_PL_058 at lat: 42.29, lon: 3.23
Processing M_PL_050 at lat: 28.27, lon: -16.61
Processing M_PL_007 at lat: 52.41, lon: 1.09
Processing M_PL_072_02 at lat: -37.73, lon: -58.23
Processing M_PL_051 at lat: -41.07, lon: -71.52
Processing M_PL_043 at lat: 56.24, lon: 9.95


In [41]:
#2015 to 2100 SSP2 temperature (monthly)

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

def process_temperature(lat, lon, start_year=2015, end_year=2100, output_csv='temperature.csv'):
    temp_ssp2_1= xr.open_dataset(f"{WORK_DIR}/Climate_data/SSP_245/tas_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_gn_20150116-21001216_v20190529.nc")



    # Extract the 'tas' variable
    temp_k = temp_ssp2_1['tas']
    
    # Clip the data for the specified period
    temp_filtered = temp_k.sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))
    
    # Change the coordinates from 0-360 to -180 to 180
    temp_filtered = temp_filtered.assign_coords(lon=(temp_filtered.lon + 180) % 360 - 180)
    temp_filtered = temp_filtered.sortby('lon')
    
    # Select the temperature value for the particular region
    temp_region = temp_filtered.sel(lat=lat, lon=lon, method='nearest')
    
    # Convert to a DataFrame
    df2 = temp_region.to_dataframe().reset_index()
    
    # Drop unnecessary columns
    df2.drop(['lat', 'lon', 'time','height'], axis=1, inplace=True)
    df2.rename(columns={'tas': 'T'}, inplace=True)
    
    # Create a DateTimeIndex starting from 'start_year-01-01' with monthly frequency
    start_date = f'{start_year}-01-01'
    df2.index = pd.date_range(start=start_date, periods=len(df2), freq='MS')
    
    # Save the resulting DataFrame as a CSV file
    df2.to_csv(output_csv, index=True)
    
    return df2

def run_for_multiple_locations(csv_file, start_year=2015, end_year=2100):
    # Read the CSV file into a DataFrame
    locations_df = pd.read_csv(csv_file)
    
    for _, row in locations_df.iterrows():
        lat = row['Latitude']
        lon = row['Longitude']
        name = row['Network ID']
        output_csv = f"{WORK_DIR}/Temperature_monthly/Temp_monthly/2015_2100_temp/AWI_{name}_SSP2.csv"
        print(f"Processing {name} at lat: {lat}, lon: {lon}")
        process_temperature(lat, lon, start_year, end_year, output_csv)

# Run the function for all locations in the CSV file
csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
run_for_multiple_locations(csv_file)


Processing M_PL_040 at lat: 18.35, lon: -77.65
Processing M_PL_030 at lat: 8.93, lon: -67.41
Processing M_PL_035 at lat: 17.92, lon: -76.19
Processing M_PL_026 at lat: -0.5, lon: -90.5
Processing M_PL_012 at lat: 28.13, lon: -17.25
Processing M_PL_058 at lat: 42.29, lon: 3.23
Processing M_PL_050 at lat: 28.27, lon: -16.61
Processing M_PL_007 at lat: 52.41, lon: 1.09
Processing M_PL_072_02 at lat: -37.73, lon: -58.23
Processing M_PL_051 at lat: -41.07, lon: -71.52
Processing M_PL_043 at lat: 56.24, lon: 9.95


In [42]:
#2015 to 2100 SSP5 temperature (monthly)

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

def process_temperature(lat, lon, start_year=2015, end_year=2100, output_csv='temperature.csv'):
    temp_ssp5_1= xr.open_dataset(f"{WORK_DIR}/Climate_data/SSP_585/tas_Amon_AWI-CM-1-1-MR_ssp585_r1i1p1f1_gn_20150116-21001216_v20190529.nc")

    # Extract the 'tas' variable
    temp_k = temp_ssp5_1['tas']
    
    # Clip the data for the specified period
    temp_filtered = temp_k.sel(time=slice(f'{start_year}-01-01', f'{end_year}-12-31'))
    
    # Change the coordinates from 0-360 to -180 to 180
    temp_filtered = temp_filtered.assign_coords(lon=(temp_filtered.lon + 180) % 360 - 180)
    temp_filtered = temp_filtered.sortby('lon')
    
    # Select the temperature value for the particular region
    temp_region = temp_filtered.sel(lat=lat, lon=lon, method='nearest')
    
    # Convert to a DataFrame
    df2 = temp_region.to_dataframe().reset_index()
    
    # Drop unnecessary columns
    df2.drop(['lat', 'lon', 'time','height'], axis=1, inplace=True)
    df2.rename(columns={'tas': 'T'}, inplace=True)
    
    # Create a DateTimeIndex starting from 'start_year-01-01' with monthly frequency
    start_date = f'{start_year}-01-01'
    df2.index = pd.date_range(start=start_date, periods=len(df2), freq='MS')
    
    # Save the resulting DataFrame as a CSV file
    df2.to_csv(output_csv, index=True)
    
    return df2

def run_for_multiple_locations(csv_file, start_year=2015, end_year=2100):
    # Read the CSV file into a DataFrame
    locations_df = pd.read_csv(csv_file)
    
    for _, row in locations_df.iterrows():
        lat = row['Latitude']
        lon = row['Longitude']
        name = row['Network ID']
        output_csv = f"{WORK_DIR}/Temperature_monthly/Temp_monthly/2015_2100_temp/AWI_{name}_SSP5.csv"
        print(f"Processing {name} at lat: {lat}, lon: {lon}")
        process_temperature(lat, lon, start_year, end_year, output_csv)

# Run the function for all locations in the CSV file
csv_file = f'{WORK_DIR}/Network_location/Network_location.csv'
run_for_multiple_locations(csv_file)


Processing M_PL_040 at lat: 18.35, lon: -77.65
Processing M_PL_030 at lat: 8.93, lon: -67.41
Processing M_PL_035 at lat: 17.92, lon: -76.19
Processing M_PL_026 at lat: -0.5, lon: -90.5
Processing M_PL_012 at lat: 28.13, lon: -17.25
Processing M_PL_058 at lat: 42.29, lon: 3.23
Processing M_PL_050 at lat: 28.27, lon: -16.61
Processing M_PL_007 at lat: 52.41, lon: 1.09
Processing M_PL_072_02 at lat: -37.73, lon: -58.23
Processing M_PL_051 at lat: -41.07, lon: -71.52
Processing M_PL_043 at lat: 56.24, lon: 9.95
